# 🎨 WIWS ComfyUI Colab (All-in-One Edition)

**Created by WIWS_MPA_C00_AI_DNA_Core_Autonomous_System**

이 노트는 **초보자 창조주님을 위해 모든 것을 자동으로 처리**합니다.
1. 구글 드라이브 연동
2. 필수 모델(Realistic Vision) 자동 다운로드
3. ComfyUI 설치 및 실행

**사용법: 위에서부터 순서대로 재생 버튼(▶)만 누르세요.**

### 1. 구글 드라이브 연동 (필수)
팝업이 뜨면 '허용'을 눌러주세요.

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

# 폴더 구조 자동 생성
base_path = '/content/drive/MyDrive/WIWS_SD_Forge'
if not os.path.exists(base_path):
    base_path = '/content/drive/MyDrive/WIWS_ComfyUI'
    os.makedirs(base_path, exist_ok=True)

os.makedirs(f'{base_path}/outputs', exist_ok=True)
os.makedirs(f'{base_path}/models', exist_ok=True)
os.makedirs(f'{base_path}/models/checkpoints', exist_ok=True)
os.makedirs(f'{base_path}/models/loras', exist_ok=True)
os.makedirs(f'{base_path}/models/vae', exist_ok=True)
os.makedirs(f'{base_path}/models/controlnet', exist_ok=True)

print(f"✅ 구글 드라이브 연결 완료! 저장 경로: {base_path}")

### 2. 필수 모델 자동 다운로드 (선물 🎁)
초보자를 위해 가장 인기 있는 실사 모델(Realistic Vision)을 자동으로 받아드립니다.
이미 있으면 건너뜁니다.

In [ ]:
import os

model_url = "https://civitai.com/api/download/models/245598"
model_path = f"{base_path}/models/checkpoints/Realistic_Vision_V6.0_B1_3D.safetensors"

if not os.path.exists(model_path):
    print("⬇️ 모델이 없어서 다운로드 중입니다... (약 2GB, 1~2분 소요)")
    !wget -c {model_url} -O "{model_path}"
    print("✅ 모델 다운로드 완료!")
else:
    print("✅ 모델이 이미 있습니다. 다운로드를 건너뜁니다.")

### 3. ComfyUI 설치 및 실행
설치가 완료되면 `trycloudflare.com` 링크를 클릭하세요.

In [ ]:
import os

%cd /content

if not os.path.exists('/content/ComfyUI'):
    !git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI

# ComfyUI Manager 설치
if not os.path.exists('/content/ComfyUI/custom_nodes/ComfyUI-Manager'):
    %cd /content/ComfyUI/custom_nodes
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git
    %cd /content/ComfyUI

# 구글 드라이브 모델 연동
!rm -rf /content/ComfyUI/models/checkpoints
!ln -s "{base_path}/models/checkpoints" /content/ComfyUI/models/checkpoints
!rm -rf /content/ComfyUI/output
!ln -s "{base_path}/outputs" /content/ComfyUI/output

# 필수 패키지 설치
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

# Cloudflared 설치
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 실행
import subprocess
import threading
import time
import socket

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared...")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("\n\033[92m🔥 접속 링크 (클릭하세요): " + l[l.find("http"):], end="\033[0m\n")

threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server